# Evaluating Scalable Bayesian Deep Learning Methods for Robust Computer Vision

The goal of this notebook is to implement and test different methods in order to make DNN more robust. First, we will import a few useful packages.

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create a directory if not exists
sample_dir = 'samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

## Robustness against class inbalance

### Loading the dataset

First let's create an imbalanced train set where the digit $9$ is very rare.

In [ ]:
batch_size = 128

data_dir = 'data'
# MNIST dataset
dataset = torchvision.datasets.MNIST(root=data_dir,
                                     train=True,
                                     transform=transforms.ToTensor(),
                                     download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
r = 0.8
mask = [True for i in range(len(dataset.targets))]
for i in range(len(dataset.targets)) :
  if dataset.targets[i].item() == 9:
    mask[i] = True if np.random.random() > r else False

In [ ]:
dataset = torch.utils.data.TensorDataset(dataset.data[mask].float(), dataset.targets[mask].byte())
train_loader = torch.utils.data.DataLoader(dataset, batch_size=256)

In [ ]:
test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(data_dir, train=False, download=True, transform=transforms.ToTensor()),
    batch_size=10, shuffle=False)

### Base model

In [ ]:
# Simple CNN Network, deterministic
class CNN_MNIST(nn.Module):
    def __init__(self, dim1 = 128, dim2 = 64):
        super(CNN_MNIST, self).__init__()
        self.cn1 = nn.Conv2d(in_channels= 1, out_channels = 64 , kernel_size=(3,3))
        self.cn2 = nn.Conv2d(in_channels= 64, out_channels = 32 , kernel_size=(3,3))
        self.mp1 = nn.MaxPool2d((3,3))

        self.fc1 = nn.Linear(2048, dim1)
        self.fc2 = nn.Linear(dim1, dim2)
        self.fc3 = nn.Linear(dim2, 10)
        self.ls = nn.LogSoftmax(1)
        
    def forward(self, x):
        y = torch.relu(self.cn1(x))
        y = torch.relu(self.cn2(y))
        y = self.mp1(y)
        y = torch.flatten(y , start_dim = 1)
        y = torch.tanh(self.fc1(y))
        y = torch.tanh(self.fc2(y))
        y = self.fc3(y)
        y = self.ls(y)
        return y

model = CNN_MNIST().to(device)
model.train()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
num_epochs = 10
# Start training
model.train()
loss = torch.nn.NLLLoss()
for epoch in range(num_epochs):
    for i, (x, y) in enumerate(train_loader):
        # Forward pass
        x = x.unsqueeze(1).to(device)
        y = y.to(device)

        y_pred = model(x)

        output = loss(y_pred, y)

        optimizer.zero_grad()
        output.backward()
        optimizer.step()
        
        if i == 0:
            print ("Epoch[{}/{}], loss: {:.4f}" 
                   .format(epoch+1, num_epochs, output.item()))

Epoch[1/10], loss: 2.3353
Epoch[2/10], loss: 0.1497
Epoch[3/10], loss: 0.0767
Epoch[4/10], loss: 0.0850
Epoch[5/10], loss: 0.0911
Epoch[6/10], loss: 0.1327
Epoch[7/10], loss: 0.0603
Epoch[8/10], loss: 0.0802
Epoch[9/10], loss: 0.0737
Epoch[10/10], loss: 0.1103


In [ ]:
model.eval()
acc_t = 0
n_t = 0
acc_t_9 = 0
n_t_9 = 0
for i, (x, y) in enumerate(train_loader):
    # Forward pass
    x = x.unsqueeze(1).to(device)
    y = y.to(device)
    y_pred = model(x)
    max_scores, max_idx_class = model(x).max(dim=1)
    acc_t += (max_idx_class == y).sum().item()
    n_t += len(y)
    acc_t_9 += (max_idx_class[y==9] == y[y==9]).sum().item()
    n_t_9 += len(y[y==9])
acc_t /= n_t
acc_t_9 /= n_t_9

print(f"Train accuracy is {acc_t}")
print(f"Train accuracy on class 9 is {acc_t_9}")
acc = 0
acc_9 = 0
n_i = 0
n_9 = 0
for i, (x, y) in enumerate(test_loader):
    # Forward pass
    x = x.to(device)
    y = y.to(device)
    y_pred = model(x)
    max_scores, max_idx_class = model(x).max(dim=1)
    acc += (max_idx_class == y).sum().item()
    n_i += len(y)
    acc_9 += (max_idx_class[y==9] == y[y==9]).sum().item()
    n_9 += len(y[y==9])
acc /= n_i
acc_9 /= n_9

print(f"Test accuracy is {acc}")
print(f"Accuracy on class 9 is {acc_9}")

Train accuracy is 0.9815250860351385
Train accuracy on class 9 is 0.9344262295081968
Test accuracy is 0.8729
Accuracy on class 9 is 0.6590683845391476


### Ensembling

In [ ]:
M = 10
num_epochs = 10
models = []

for m in range(M):
  # Start training
  model = CNN_MNIST().to(device)
  model.train()
  optimizer = torch.optim.Adam(model.parameters())
  loss = torch.nn.NLLLoss()
  for epoch in range(num_epochs):
    for i, (x, y) in enumerate(train_loader):
      # Forward pass
      x = x.unsqueeze(1).to(device)
      y = y.to(device)

      y_pred = model(x)

      output = loss(y_pred, y)

      optimizer.zero_grad()
      output.backward()
      optimizer.step()
        
      if i == 0:
        print ("Epoch[{}/{}], loss: {:.4f}" .format(epoch+1, num_epochs, output.item()))
  models.append(model)

Epoch[1/10], loss: 2.3834
Epoch[2/10], loss: 0.1428
Epoch[3/10], loss: 0.0881
Epoch[4/10], loss: 0.0787
Epoch[5/10], loss: 0.0871
Epoch[6/10], loss: 0.0817
Epoch[7/10], loss: 0.0931
Epoch[8/10], loss: 0.1002
Epoch[9/10], loss: 0.1266
Epoch[10/10], loss: 0.0839
Epoch[1/10], loss: 2.3408
Epoch[2/10], loss: 0.1308
Epoch[3/10], loss: 0.1075
Epoch[4/10], loss: 0.0682
Epoch[5/10], loss: 0.0678
Epoch[6/10], loss: 0.0732
Epoch[7/10], loss: 0.0587
Epoch[8/10], loss: 0.0676
Epoch[9/10], loss: 0.0984
Epoch[10/10], loss: 0.0474
Epoch[1/10], loss: 2.4021
Epoch[2/10], loss: 0.1276
Epoch[3/10], loss: 0.0754
Epoch[4/10], loss: 0.1180
Epoch[5/10], loss: 0.0783
Epoch[6/10], loss: 0.1035
Epoch[7/10], loss: 0.0942
Epoch[8/10], loss: 0.0701
Epoch[9/10], loss: 0.0634
Epoch[10/10], loss: 0.0624
Epoch[1/10], loss: 2.3046
Epoch[2/10], loss: 0.1065
Epoch[3/10], loss: 0.0816
Epoch[4/10], loss: 0.0956
Epoch[5/10], loss: 0.0670
Epoch[6/10], loss: 0.0898
Epoch[7/10], loss: 0.0617
Epoch[8/10], loss: 0.0722
Epoch[9/1

In [ ]:
m = models[0]
models[0] = models[1]
models[1] = m

In [ ]:
acc_t = 0
n_t = 0
acc_t_9 = 0
n_t_9 = 0
K = 10

for i, (x, y) in enumerate(train_loader):
    # Forward pass
    x = x.unsqueeze(1).to(device)
    y = y.to(device)
    y_pred = models[0](x)
    for j in range(1, K):
      y_pred += models[j](x)
    max_scores, max_idx_class = y_pred.max(dim=1)
    acc_t += (max_idx_class == y).sum().item()
    n_t += len(y)
    acc_t_9 += (max_idx_class[y==9] == y[y==9]).sum().item()
    n_t_9 += len(y[y==9])
acc_t /= n_t
acc_t_9 /= n_t_9

print(f"Train accuracy is {acc_t}")
print(f"Train accuracy on class 9 is {acc_t_9}")
acc = 0
acc_9 = 0
n_i = 0
n_9 = 0
for i, (x, y) in enumerate(test_loader):
    # Forward pass
    x = x.to(device)
    y = y.to(device)
    y_pred = models[0](x)
    for j in range(1, K):
      y_pred += models[j](x)
    max_scores, max_idx_class = y_pred.max(dim=1)
    acc += (max_idx_class == y).sum().item()
    n_i += len(y)
    acc_9 += (max_idx_class[y==9] == y[y==9]).sum().item()
    n_9 += len(y[y==9])
acc /= n_i
acc_9 /= n_9

print(f"Test accuracy is {acc}")
print(f"Accuracy on class 9 is {acc_9}")

Train accuracy is 0.9912199713970202
Train accuracy on class 9 is 0.9444444444444444
Test accuracy is 0.967
Accuracy on class 9 is 0.9078295341922695


On obtient les valeurs suivantes :

| M | Train Acc | 9-train acc | Test Acc | 9-Test acc |
|-------|-------|--------|-------|-------|
| 1 | 0.981 | 0.914 | 0.709 | 0.573 |
| 2 | 0.986 | 0.928 | 0.95 | 0.882 |
| 4 | 0.989 | 0.935 | 0.969 | 0.926 |
| 6 | 0.99 | 0.942 | 0.971 | 0.907 |
| 8 | 0.99 | 0.939 | 0.967 | 0.918 |
| 10 | 0.991 | 0.944 | 0.967 | 0.908 |

### MC-Dropout

In [ ]:
# Simple CNN Network, deterministic
class dropout_MNIST(nn.Module):
    def __init__(self, dim1 = 128, dim2 = 64, p_d = 0.2):
        super(dropout_MNIST, self).__init__()
        self.cn1 = nn.Conv2d(in_channels= 1, out_channels = 64 , kernel_size=(3,3))
        self.dr1 = nn.Dropout2d(p = p_d)
        self.cn2 = nn.Conv2d(in_channels= 64, out_channels = 32 , kernel_size=(3,3))
        self.dr2 = nn.Dropout2d(p = p_d)
        self.mp1 = nn.MaxPool2d((3,3))

        self.fc1 = nn.Linear(2048, dim1)
        self.fc2 = nn.Linear(dim1, dim2)
        self.fc3 = nn.Linear(dim2, 10)
        self.ls = nn.LogSoftmax(1)
        
    def forward(self, x):
        y = torch.relu(self.cn1(x))
        y = self.dr1(y)
        y = torch.relu(self.cn2(y))
        y = self.dr2(y)
        y = self.mp1(y)
        y = torch.flatten(y , start_dim = 1)
        y = torch.tanh(self.fc1(y))
        y = torch.tanh(self.fc2(y))
        y = self.fc3(y)
        y = self.ls(y)
        return y

model = dropout_MNIST().to(device)
model.train()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
num_epochs = 20
# Start training
model.train()
loss = torch.nn.NLLLoss()
for epoch in range(num_epochs):
    for i, (x, y) in enumerate(train_loader):
        # Forward pass
        x = x.unsqueeze(1).to(device)
        y = y.to(device)

        y_pred = model(x)

        output = loss(y_pred, y)

        optimizer.zero_grad()
        output.backward()
        optimizer.step()
        
        if i == 0:
            print ("Epoch[{}/{}], loss: {:.4f}" 
                   .format(epoch+1, num_epochs, output.item()))

Epoch[1/20], loss: 2.3299
Epoch[2/20], loss: 0.2106
Epoch[3/20], loss: 0.1721
Epoch[4/20], loss: 0.1353
Epoch[5/20], loss: 0.1167
Epoch[6/20], loss: 0.1663
Epoch[7/20], loss: 0.1719
Epoch[8/20], loss: 0.1931
Epoch[9/20], loss: 0.1012
Epoch[10/20], loss: 0.1254
Epoch[11/20], loss: 0.1648
Epoch[12/20], loss: 0.1511
Epoch[13/20], loss: 0.1414
Epoch[14/20], loss: 0.1122
Epoch[15/20], loss: 0.0884
Epoch[16/20], loss: 0.1178
Epoch[17/20], loss: 0.0926
Epoch[18/20], loss: 0.1287
Epoch[19/20], loss: 0.1282
Epoch[20/20], loss: 0.1542


In [ ]:
acc_t = 0
n_t = 0
acc_t_9 = 0
n_t_9 = 0
K = 10

for i, (x, y) in enumerate(train_loader):
    # Forward pass
    x = x.unsqueeze(1).to(device)
    y = y.to(device)
    y_pred = model(x)
    for j in range(1, K):
      y_pred += model(x)
    max_scores, max_idx_class = y_pred.max(dim=1)
    acc_t += (max_idx_class == y).sum().item()
    n_t += len(y)
    acc_t_9 += (max_idx_class[y==9] == y[y==9]).sum().item()
    n_t_9 += len(y[y==9])
acc_t /= n_t
acc_t_9 /= n_t_9

print(f"Train accuracy is {acc_t}")
print(f"Train accuracy on class 9 is {acc_t_9}")
acc = 0
acc_9 = 0
n_i = 0
n_9 = 0
for i, (x, y) in enumerate(test_loader):
    # Forward pass
    x = x.to(device)
    y = y.to(device)
    y_pred = model(x)
    for j in range(1, K):
      y_pred += model(x)
    max_scores, max_idx_class = y_pred.max(dim=1)
    acc += (max_idx_class == y).sum().item()
    n_i += len(y)
    acc_9 += (max_idx_class[y==9] == y[y==9]).sum().item()
    n_9 += len(y[y==9])
acc /= n_i
acc_9 /= n_9

print(f"Test accuracy is {acc}")
print(f"Accuracy on class 9 is {acc_9}")

Train accuracy is 0.9763934901066276
Train accuracy on class 9 is 0.8636363636363636
Test accuracy is 0.9574
Accuracy on class 9 is 0.8959365708622399



| M | Train Acc | 9-train acc | Test Acc | 9-Test acc |
|-------|-------|--------|-------|-------|
| 1 | 0.964 | 0.823 | 0.940 | 0.869 |
| 2 | 0.972 | 0.856 | 0.952 | 0.889 |
| 4 | 0.975 | 0.896 | 0.956 | 0.896 |
| 6 | 0.976 | 0.873 | 0.956 | 0.895 |
| 8 | 0.976 | 0.867 | 0.957 | 0.896 |
| 10 | 0.976 | 0.863 | 0.957 | 0.896 |